In [0]:
!pip uninstall tensorflow

In [0]:
# !pip install tensorflow==2.0.0
!pip install tensorflow-gpu

In [0]:
import tensorflow as tf
import numpy as np

In [23]:
tf.__version__

'2.0.0'

In [58]:
if tf.test.is_gpu_available():
    print("\nYour GPU is available to TensorFlow\n")
else:
    print("\nYour GPU is NOT available to TensorFlow\n")



Your GPU is available to TensorFlow



In [24]:
print(('Is your GPU available for use?\n{0}').format(
    'Yes, your GPU is available: True' if tf.test.is_gpu_available() == True else 'No, your GPU is NOT available: False'
))


Is your GPU available for use?
Yes, your GPU is available: True


In [0]:
from tensorflow.keras.layers import AveragePooling2D, Conv2D
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import RandomNormal
import matplotlib.pyplot as plt
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score


In [0]:
# Load and prepare the MNIST dataset-
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
# Normalize and convert samples from integers to floating-point numbers-
X_train, X_test = X_train / 255.0, X_test / 255.0

X_train = tf.cast(X_train, dtype=tf.float32)
X_test = tf.cast(X_test, dtype=tf.float32)


In [0]:
# Perform one-hot encoding for targets-
n_classes = 10

y_train_ohe = tf.keras.utils.to_categorical(y_train, n_classes)
y_test_ohe = tf.keras.utils.to_categorical(y_test, n_classes)


In [29]:
# Reshape training and testing data-
X_train = tf.reshape(X_train, shape=(X_train.shape[0], 28, 28, 1))
X_test = tf.reshape(X_test, shape = (X_test.shape[0], 28, 28, 1))


print("\nX_train.shape = {0} & y_train_ohe.shape = {1}".format(X_train.shape, y_train_ohe.shape))
print("X_test.shape = {0} & y_test_ohe.shape = {1}\n".format(X_test.shape, y_test_ohe.shape))


X_train.shape = (60000, 28, 28, 1) & y_train_ohe.shape = (60000, 10)
X_test.shape = (10000, 28, 28, 1) & y_test_ohe.shape = (10000, 10)



In [0]:
weight_init = RandomNormal()

batch_size = 128
epochs = 50

In [0]:
# Define the LeNet-5 CNN model for MNIST classification:

model = Sequential()

# Convolutional layer-1
model.add(Conv2D(filters = 6, kernel_size = (5, 5),
    strides = (1, 1), activation = 'relu', input_shape = (28, 28, 1)))

# Average pooling layer-1
model.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2)))

# Convolutional layer-2
model.add(Conv2D(filters = 16, kernel_size = (5, 5),
    strides = (1, 1), activation = 'relu'))

# Average pooling layer-2
model.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2)))

# Flatten the output-
model.add(layers.Flatten())

# Fully connected layer-
model.add(layers.Dense(units = 120, activation = 'relu'))

# Another flattening of the previous layer-
model.add(layers.Flatten())

# Fully connected layer-
model.add(layers.Dense(units = 84, activation = 'relu'))

# Output layer-
model.add(layers.Dense(units = 10, activation = 'softmax'))


In [0]:
# Define an optimizer for the model to choose-
optimizer_SGD = tf.keras.optimizers.SGD(lr = 0.001,
	momentum=0.0, decay = 0.0)


In [0]:
# Compile the designed nn model-
model.compile(loss = 'categorical_crossentropy',
	optimizer=optimizer_SGD, metrics=['accuracy'])


In [0]:
@tf.function
def step(X_train, y_train):

  with tf.GradientTape() as tape:
    # Make prediction-
    y_pred = model(X_train)

    # Compute error/loss-
    model_loss = tf.keras.losses.categorical_crossentropy(y_true=y_train, y_pred=y_pred)

    # print("type(model_loss) = {0} & model_loss.shape = {1}".format(type(model_loss), model_loss.shape))
    # type(model_loss) = <class 'tensorflow.python.framework.ops.Tensor'> & model_loss.shape = (128,)

    # print(model_loss[:5])
    # print("cost = {0:.4f}".format(tf.math.reduce_mean(model_loss)))

  # Compute gradients-
  model_gradients = tape.gradient(model_loss, model.trainable_variables)

  # Update parameters of model-
  optimizer_SGD.apply_gradients(zip(model_gradients, model.trainable_variables))



In [56]:
# Training loop-
batch_per_epoch = math.floor(len(X_train) / batch_size)

for epoch in range(epochs):
	print("\ncurrent epoch = {0}\n".format(epoch))

	for i in range(batch_per_epoch):
		n = i * batch_size
		# step(X_train[n: batch_size + n], y_train_ohe[n: batch_size + n])
		step(X_train[n: n+batch_size], y_train_ohe[n: n + batch_size])
  


current epoch = 0


current epoch = 1


current epoch = 2


current epoch = 3


current epoch = 4


current epoch = 5


current epoch = 6


current epoch = 7


current epoch = 8


current epoch = 9


current epoch = 10


current epoch = 11


current epoch = 12


current epoch = 13


current epoch = 14


current epoch = 15


current epoch = 16


current epoch = 17


current epoch = 18


current epoch = 19


current epoch = 20


current epoch = 21


current epoch = 22


current epoch = 23


current epoch = 24


current epoch = 25


current epoch = 26


current epoch = 27


current epoch = 28


current epoch = 29


current epoch = 30


current epoch = 31


current epoch = 32


current epoch = 33


current epoch = 34


current epoch = 35


current epoch = 36


current epoch = 37


current epoch = 38


current epoch = 39


current epoch = 40


current epoch = 41


current epoch = 42


current epoch = 43


current epoch = 44


current epoch = 45


current epoch = 46


current epoch = 47


c

In [57]:
# Evaluate trained model-
model.evaluate(X_test, y_test_ohe)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.07075431137831698, 0.9886]

In [59]:
# Make predictions using trained model-
y_pred = model.predict_classes(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')

print("\nMetrics of trained CNN are:")
print("accuracy = {0:.4f}, precision = {1:.4f} & recall = {2:.4f}\n".format(accuracy, precision, recall))


Metrics of trained CNN are:
accuracy = 0.9886, precision = 0.9885 & recall = 0.9884

